<a href="https://colab.research.google.com/github/migperfer/MIR-UPF/blob/master/best_mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download and import the python packages

In [0]:
!pip install  essentia numpy matplotlib
import os
import numpy as np
import matplotlib as plt
import essentia.standard as std
import pandas as pd
if not os.path.isdir('TIVlib'):
    !git clone https://github.com/aframires/TIVlib
else:
    print("TIVlib already installed")
from TIVlib import TIVlib
from glob import glob
from IPython.display import display, Audio, HTML
import re
import csv
from zipfile import ZipFile
import requests

## Download and extract the folder contaning the audio loops

In [0]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
file_id = '1mS0G_Gk4v6IHe2E2IZ2Z6dcE-lcTuZx3'
destination = 'audios.zip'
download_file_from_google_drive(file_id, destination)

In [0]:
# Unzip the file
with ZipFile('audios.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall()

## Get all the mp3 files and select a target audio

#### Retrieve all possible loops

In [0]:
mp3list = glob('subset/*.mp3')

## Define a function to get beatwise TIVs

In [0]:
def get_beat_chunks(filename, bpm_restrict=None):
    audio = std.MonoLoader(filename=filename)()
    hpcp = std.HPCP()
    spectrum = std.Spectrum()
    speaks = std.SpectralPeaks()
    tivs = []
    sr = 44100
    bpm = get_tempo(filename)
    tivs_framewise = []
    if bpm_restrict != None and bpm_restrict!=bpm:
        raise ValueError
    sec_beat = (60/bpm)
    beats = np.arange(0, len(audio)/sr, sec_beat)
    beats = np.append(beats, len(audio)/sr)
    frames = []
    for i in range(1, len(beats)):
        segmented_audio = audio[int(beats[i - 1] * sr):int(beats[i] * sr)]
        cutter = std.FrameGenerator(segmented_audio)
        aux = []
        for sec in cutter:
            spec = spectrum(sec)
            freq, mag = speaks(spec)
            chroma = hpcp(freq, mag)
            tivs_framewise.append(TIVlib.TIV.from_pcp(chroma))
            aux.append(chroma)
            frames.append(spec)
        chroma = np.mean(aux, axis=1)
        tiv = TIVlib.TIV.from_pcp(chroma)
        tivs.append(tiv)
    
    # Calculate the whole TIV
    frame_mean = np.mean(frames, axis=1)
    freq, mag = speaks(frame_mean)
    chroma_whole = hpcp(freq, mag)
    tiv_whole = TIVlib.TIV.from_pcp(chroma_whole)
    return tivs, tiv_whole, tivs_framewise

def get_number_beats(filename):
    audio = std.MonoLoader(filename=filename)()
    sr = 44100
    bpm = get_tempo(filename)
    sec_beat = (60/bpm)
    beats = np.arange(0, len(audio)/sr, sec_beat)
    beats = np.append(beats, len(audio)/sr)
    return len(beats)

def get_tempo(filename):
    try:
        bpm = int(re.search(r"(\d+)bpm", filename).group(1))
    except:
        bpm = int(re.search(r"/(\d+)-", filename).group(1))
    return bpm

## Define functions to retrieve Essentia dissonance

In [0]:

def audio_dissonance(filename1, filename2):
    audio1 = std.MonoLoader(filename=filename1)()
    audio2 = std.MonoLoader(filename=filename2)()
    spectrum = std.Spectrum()
    speaks = std.SpectralPeaks()
    diss = std.Dissonance()
    dissonances = []
    sr = 44100
    bpm1 = get_tempo(filename1)
    bpm2 = get_tempo(filename2)
    if bpm1 != bpm2:
      raise ValueError("Different tempo")
    else:
      bpm = bpm1
    sec_beat = (60/bpm)
    beats = np.arange(0, len(audio2)/sr, sec_beat)
    beats = np.append(beats, len(audio2)/sr)
    aux_total = []
    for i in range(1, len(beats)):
        # Beatwise dissonance
        segmented_audio1 = audio1[int(beats[i - 1] * sr):int(beats[i] * sr)]
        segmented_audio2 = audio2[int(beats[i - 1] * sr):int(beats[i] * sr)]
        try:
          segmented_audio = segmented_audio1 + segmented_audio2[:len(segmented_audio1)]
        except:
          segmented_audio = segmented_audio2 + segmented_audio1[:len(segmented_audio2)]
        cutter = std.FrameGenerator(segmented_audio)
        aux = []
        for sec in cutter:
            spec = spectrum(sec)
            aux_total.append(spec)
            freq, mag = speaks(spec)
            aux.append(diss(freq, mag))  # Framewise dissonance for beat i
        dissonances.append(np.mean(aux)) # Append mean dissonance for beat i
    
    # Whole dissonance
    freq, mag = speaks(np.mean(aux_total, axis=1))
    dissonance = diss(freq, mag)  # Dissonance produced by the mean spectrum
    return np.mean(dissonances), dissonance

def pad_zeros(arr):
  arr_len = np.log2(len(arr))
  fin_len = np.ceil(arr_len)
  deficit = int(np.power(2, fin_len) - len(arr))
  arr = np.concatenate((np.zeros(deficit, dtype=arr.dtype), arr))
  return arr

## Get an overview of the dataset

In [0]:
df = []
if not os.path.isfile('compt_loops.csv'):
  for song in mp3list:
    df.append({'filename': song, 'n_beats': get_number_beats(song), 'tempo': get_tempo(song)})
  df = pd.DataFrame(df)
  print("Before trimming: %s" % len(df))
  final_df = df[~df.filename.str.contains('(percussion|drums|beat|sub-bass|kick|hats|snare|-808-|-808.mp3|-sub-|-sub.mp3|hihat|percs)', regex=True)]
  idx = final_df.groupby(['n_beats','tempo']).count().idxmax()
  n_beats_max = idx[0][0]
  tempo_max = idx[0][1]
  final_df_idx = (final_df['n_beats'] == n_beats_max) & (final_df['tempo'] == tempo_max)
  final_df = final_df[final_df_idx]
  print("Maximum number of common feature samples for %s number of beats and %s bpm. A total of %s loops" % 
        (n_beats_max, tempo_max, len(final_df)))
  mp3list = final_df['filename'].tolist()
  final_df.to_csv('compt_loops.csv')

### Select a **target audio** randomly
Listen to the target loop. If it's mainly a drum loop, rerun this cell until you get something that doesn't contain mainly percussion.



In [0]:
# Target audio selected using the initial form
loop_sample = np.random.randint(0, len(mp3list)-1)
target_audio = mp3list[loop_sample]
display(Audio(filename=target_audio))
main_song_tivs, main_song_tiv_whole, main_song_framewise = get_beat_chunks(target_audio)

## Load all the compatible loops
And then calculate the compatibilities


In [0]:
filenames = []
with open('compt_loops.csv', 'r') as file:
    dicw = csv.DictReader(file)
    for row in dicw:
        filenames.append(row['filename'])

### TIV

In [0]:
compdict = {}  # A dictionary to keep all compatibilities
compdict_whole = {}  # A dictionary to keep all compatibilities (whole version)
compdict_framewise = {}
for candidate in filenames:
    tivcand, tivcand_whole, tivframewise = get_beat_chunks(candidate)
    # TIV beatwise
    comp = []
    for i in range(len(main_song_tivs)):
        comp.append(main_song_tivs[i].small_scale_compatibility(tivcand[i]))
    compdict[candidate] = np.sum(comp)
    # TIV Whole
    compdict_whole[candidate] = main_song_tiv_whole.small_scale_compatibility(tivcand_whole)
    # TIV Framewise
    comp = []
    for i in range(len(tivframewise)):
      comp.append(main_song_framewise[i].small_scale_compatibility(tivframewise[i]))
    compdict_framewise[candidate] = np.sum(comp)

### Essentia Dissonance

In [0]:
compdict_diss = {}  # A dictionary to keep all compatibilities
for candidate in filenames:
  framewise_diss, whole_diss = audio_dissonance(candidate, target_audio)
  compdict_diss[candidate] = framewise_diss

## Sort the loops according to compatibility
Create also a dictionary containing the 10 most compatibles

#### TIV

In [0]:
#Beatwise version
compdict = {k: v for k, v in sorted(compdict.items(), key=lambda item: item[1], reverse=True)}
dict_10 = {}
for x in list(compdict)[1:11]:
    dict_10[x] = compdict[x]

In [0]:
#Whole version
compdict_whole = {k: v for k, v in sorted(compdict_whole.items(), key=lambda item: item[1], reverse=True)}
dict_10_w = {}
for x in list(compdict_whole)[1:11]:
    dict_10_w[x] = compdict_whole[x]

In [0]:
#Framewise version
compdict_framewise = {k: v for k, v in sorted(compdict_framewise.items(), key=lambda item: item[1], reverse=True)}
dict_10_f = {}
for x in list(compdict_framewise)[1:11]:
    dict_10_f[x] = compdict_framewise[x]

#### Essentia dissonance

In [0]:
#Framewise mean version
compdict_diss = {k: v for k, v in sorted(compdict_diss.items(), key=lambda item: item[1],  reverse=True)}
dict_10_d = {}
for x in list(compdict_diss)[1:11]:
    dict_10_d[x] = compdict_diss[x]

#### Some utilities

In [0]:
def intersection(lst1, lst2): 
    intersect_list = []
    for el1 in lst1:
      if el1 in lst2:
        intersect_list.append(el1) 
    return intersect_list 

intersection_tivbeatwise_tivwhole = len(intersection(list(dict_10_w.keys()), list(dict_10.keys())))*10
intersection_tivbeatwise_dissonan = len(intersection(list(dict_10_d.keys()), list(dict_10.keys())))*10
intersection_tiv_dissonance = len(intersection(list(dict_10_w.keys()), list(dict_10_d.keys())))*10

In [0]:
def create_mix(song1, song2):
    audio1 = std.MonoLoader(filename=song1)()
    audio2 = std.MonoLoader(filename=song2)()
    audio1 = audio1/max(audio1)/2
    audio2 = audio2/max(audio2)/2
    try:
      mix = audio1 + audio2[:len(audio1)]
    except:
      mix = audio2 + audio1[:len(audio2)]
    return mix

## Create the mixes and listen to them (TIV Beatwise)
This will output the ten most compatible **candidate loops**  along with the mixes with the **target loops**. Those mixes will be created with the beatwise TIV.

In [0]:
i = 0
for key in dict_10.keys():
    display(HTML("<h3>Mix #%s: %s</h3>" % (i, key.split('/')[-1])))
    display(HTML("Original"))
    display(Audio(filename=key))
    mix = create_mix(target_audio, key)
    display(HTML("Mix"))
    display(Audio(data=mix, rate=44100))
    i += 1

## Create the mixes and listen to them (TIV Whole)
This will output the ten most compatible **candidate loops**  along with the mixes with the **target loops**. Those mixes will be created with the whole TIV.

In [0]:
i = 0
for key in dict_10_w.keys():
    display(HTML("<h3>Mix #%s: %s</h3>" % (i, key.split('/')[-1])))
    display(HTML("Original"))
    display(Audio(filename=key))
    mix = create_mix(target_audio, key)
    display(HTML("Mix"))
    display(Audio(data=mix, rate=44100))
    i += 1

## Create the mixes and listen to them (TIV framewise)

In [0]:
i = 0
for key in dict_10_f.keys():
    display(HTML("<h3>Mix #%s: %s</h3>" % (i, key.split('/')[-1])))
    display(HTML("Original"))
    display(Audio(filename=key))
    mix = create_mix(target_audio, key)
    display(HTML("Mix"))
    display(Audio(data=mix, rate=44100))
    i += 1

## Create the mixes and listen to them (Dissonance)



In [0]:
i = 0
for key in dict_10_d.keys():
    display(HTML("<h3>Mix #%s: %s</h3>" % (i, key.split('/')[-1])))
    display(HTML("Original"))
    display(Audio(filename=key))
    mix = create_mix(target_audio, key)
    display(HTML("Mix"))
    display(Audio(data=mix, rate=44100))
    i += 1